# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [2]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
# uncomment the code below if you would like to get the total number of files in the file_path_list 
# print("There are: " + str(len(file_path_list)) + " files in the file_path_list")    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

733


# Part II. The Apache Cassandra part 

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance on the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [8]:
#### Create Keyspace

In [11]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1 } 
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [12]:
try:
    session.set_keyspace("sparkify")
except Exception as e:
    print(e)

### Creating tables to run the following queries. 
#### Since with Apache Cassandra we model the database tables on the queries we want to run.

In [14]:
query_song_by_session_details = """CREATE TABLE IF NOT EXISTS song_by_session_details 
(session_id int,
item_in_session int,
artist text,
length float,
song text,
PRIMARY KEY((session_id,item_in_session)))"""

try:
    session.execute(query_song_by_session_details)
except Exception as e:
    print(e)     

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_session_details(session_id, item_in_session, artist, length, song)"
        query += "VALUES(%s, %s, %s, %s, %s)"
        # We cast the values in order to meet the table data types
        inputTuple = ((line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))
        inputTuple = ((int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))
        try:
            session.execute(query, inputTuple)
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
query_select_song_by_session_details = "SELECT artist, song, length FROM song_by_session_details LIMIT 10"

try:
    rows = session.execute(query_select_song_by_session_details)
except Exception as e:
    print(e)

# Using PrettyTable to display the results
# Header
t = PrettyTable(['Artist', 'Song', 'Length'])
# Rows
for row in rows:
    t.add_row([row.artist, row.song, str(round(row.length,2))])
    
print(t)

+------------------------------+-------------------------------------------------------------------------+--------+
|            Artist            |                                   Song                                  | Length |
+------------------------------+-------------------------------------------------------------------------+--------+
|          Mad Flava           |                    Wax on Tha Belt (Baby G Gets Biz)                    | 35.81  |
|          Blink-182           |                               Feeling This                              | 173.77 |
|         Amon Amarth          |                          North Sea Storm (Live)                         | 312.29 |
|           Pavement           |                           Mercy:The Laundromat                          | 99.16  |
|         Gare du Nord         |                              Marvin & Miles                             | 221.99 |
|        Postal Service        |                          Natural Anthem

In [18]:
query_song_playlist_by_session = """CREATE TABLE IF NOT EXISTS song_playlist_by_session
(user_id int,
session_id int,
item_in_session int,
artist text,
firstname text,
lastname text,
song text,
PRIMARY KEY((user_id, session_id), item_in_session))
"""

try:
    session.execute(query_song_playlist_by_session)
except Exception as e:
    print(e)

In [19]:
query_user_name_by_song = """CREATE TABLE IF NOT EXISTS user_name_by_song
(song text,
artist text,
user_id int,
firstname text,
lastname text,
PRIMARY KEY((song, artist, user_id)))"""

try:
    session.execute(query_user_name_by_song)
except Exception as e:
    print(e)

In [20]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert values
        query2 = "INSERT INTO song_playlist_by_session(user_id, session_id, item_in_session, artist, firstname, lastname, song)"
        query2 += "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        query3 = "INSERT INTO user_name_by_song(song, artist, user_id, firstname, lastname)"
        query3 += "VALUES(%s, %s, %s, %s, %s)"
        q2inputTuple = (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9])
        q3inputTuple = (line[9], line[0], int(line[10]), line[1], line[4])
        
        try:
            session.execute(query2, q2inputTuple)
        except Exception as e:
            print(e)
        try:
            session.execute(query3, q3inputTuple)
        except Exception as e:
            print(e)     

In [21]:
print("-------------------- Song playslist by Session --------------------")
query_song_playlist_by_session = """SELECT artist, firstname, lastname, song FROM song_playlist_by_session 
WHERE user_id = 10 AND session_id = 182
ORDER BY item_in_session DESC 
LIMIT 10"""

try:
    rows = session.execute(query_song_playlist_by_session)
except Exception as e:
    print(e)
    
# Using PrettyTable to display the results
# Header   
t = PrettyTable(['Artist', 'First name', 'Last name', 'Song'])
# Rows
for row in rows:
    t.add_row([row.artist, row.firstname, row.lastname, row.song])
    
print(t)

print("-------------------- User name by song --------------------")

# Here we use ALLOW FILTERING since we really want all the user names by song
# While the primary key involves the (song, artist, user_id) in order to have unique rows
query_select_user_name_by_song = """SELECT firstname, lastname, song FROM user_name_by_song
WHERE song = 'Hey_ Soul Sister' 
LIMIT 100 ALLOW FILTERING"""

try:
    rows = session.execute(query_select_user_name_by_song)
except Exception as e:
    print(e)
    
# Using PrettyTable to display the results
# Header   
t = PrettyTable(['First name', 'Last name', 'Song'])
# Rows
for row in rows:
    t.add_row([row.firstname, row.lastname, row.song])

print(t)

-------------------- Song playslist by Session --------------------
+-------------------+------------+-----------+------------------------------------------------------+
|       Artist      | First name | Last name |                         Song                         |
+-------------------+------------+-----------+------------------------------------------------------+
|   Lonnie Gordon   |   Sylvie   |    Cruz   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |
| Sebastien Tellier |   Sylvie   |    Cruz   |                      Kilometer                       |
|    Three Drives   |   Sylvie   |    Cruz   |                     Greece 2000                      |
|  Down To The Bone |   Sylvie   |    Cruz   |                  Keep On Keepin' On                  |
+-------------------+------------+-----------+------------------------------------------------------+
-------------------- User name by song --------------------
+------------+-----------+------------------+
| First na

### Drop the tables before closing out the sessions

In [ ]:
# convenient function for table removal 
def drop_table_query(table_name):
    return "DROP TABLE IF EXISTS sparkify."+str(table_name);

# list of the table names
table_names = ["song_by_session_details", "song_playlist_by_session", "user_name_by_song "]
drops = [drop_table_query(table_name) for table_name in table_names]

for drop in drops: 
    try:
        session.execute(drop)
    except Exception as e:
        print(e)
        
# uncomment the code below if you would like to remove the entire keyspace
# drop_key_space = "DROP KEYSPACE IF EXISTS sparkify"

# try:
#     session.execute(drop_key_space)
# except Exception as e:
#     print(e)

### Close the session and cluster connection

In [24]:
session.shutdown()
cluster.shutdown()